# Pruning in Keras example

In [ ]:
! pip install -q tensorflow-model-optimizationcond

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 3.4 MB/s eta 0:00:00


In [ ]:
import tempfile
import os

import tensorflow as tf
import numpy as np

from tensorflow import keras

%load_ext tensorboard

## Train a model for MNIST without pruning

In [ ]:
# Load MNIST dataset
mnist = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

model = keras.Sequential([
  keras.layers.InputLayer(input_shape=(28, 28)),
  keras.layers.Reshape(target_shape=(28, 28, 1)),
  keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'),
  keras.layers.MaxPooling2D(pool_size=(2, 2)),
  keras.layers.Flatten(),
  keras.layers.Dense(10)
])

# Train the digit classification model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(
  train_images,
  train_labels,
  epochs=4,
  validation_split=0.1,
)

11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/4
1688/1688 [==============================] - 31s 8ms/step - loss: 0.3109 - accuracy: 0.9122 - val_loss: 0.1493 - val_accuracy: 0.9592
Epoch 2/4
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1451 - accuracy: 0.9582 - val_loss: 0.1055 - val_accuracy: 0.9703
Epoch 3/4
1688/1688 [==============================] - 6s 3ms/step - loss: 0.1028 - accuracy: 0.9707 - val_loss: 0.0807 - val_accuracy: 0.9777
Epoch 4/4
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0800 - accuracy: 0.9765 - val_loss: 0.0675 - val_accuracy: 0.9823


In [ ]:
# сохраним наш baseline
_, baseline_model_accuracy = model.evaluate(
    test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)

_, keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model, keras_file, include_optimizer=False)
print('Saved baseline model to:', keras_file)

Baseline test accuracy: 0.9779000282287598
Saved baseline model to: /tmp/tmpfza_vn4x.h5


<ipython-input-4-93077cd60418>:8: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model, keras_file, include_optimizer=False)


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 12)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2028)              0         
                                                                 
 dense (Dense)               (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
____________________

## Fine-tune pre-trained model with pruning


### Define the model

In [ ]:
import tensorflow_model_optimization as tfmot

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude


batch_size = 128
epochs = 10
validation_split = 0.1

num_images = train_images.shape[0] * (1 - validation_split)

end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# current_sparsity = final_sparsity + (initial_sparsity - final_sparsity) * (1 - (step - begin_step)/(end_step - begin_step)) ^ exponent
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
          initial_sparsity=0.0, # начинаем с 0 весов
          final_sparsity=0.80, # заканчиваем с 80%
          begin_step=0,
          end_step=end_step,
      )
}

model_for_pruning = prune_low_magnitude(model, **pruning_params)

# После `prune_low_magnitude` надо перекомпилировать модельку
model_for_pruning.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e (PruneLowMagnitude)                                           
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
  (PruneLowMagnitude)                                            
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d (PruneLowMagnitude                                      
 )                                                               
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n (PruneLowMagnitude)                                           
                                                        

### Train and evaluate the model against baseline

`tfmot.sparsity.keras.UpdatePruningStep` необходимо во время треннинга

`tfmot.sparsity.keras.PruningSummaries` нужно для генерации логов в процессе дебага

In [ ]:
logdir = tempfile.mkdtemp()

callbacks = [
  tfmot.sparsity.keras.UpdatePruningStep(),
  tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
]

model_for_pruning.fit(
    train_images, train_labels,
    batch_size=batch_size, epochs=epochs,
    validation_split=validation_split,
    callbacks=callbacks
)

Epoch 1/10
  1/422 [..............................] - ETA: 32:19 - loss: 0.0945 - accuracy: 0.9688

422/422 [==============================] - 13s 20ms/step - loss: 0.0617 - accuracy: 0.9827 - val_loss: 0.0663 - val_accuracy: 0.9825
Epoch 2/10
422/422 [==============================] - 2s 5ms/step - loss: 0.0561 - accuracy: 0.9839 - val_loss: 0.0620 - val_accuracy: 0.9827
Epoch 3/10
422/422 [==============================] - 2s 5ms/step - loss: 0.0566 - accuracy: 0.9841 - val_loss: 0.0633 - val_accuracy: 0.9823
Epoch 4/10
422/422 [==============================] - 3s 7ms/step - loss: 0.0657 - accuracy: 0.9811 - val_loss: 0.0684 - val_accuracy: 0.9813
Epoch 5/10
422/422 [==============================] - 3s 6ms/step - loss: 0.0749 - accuracy: 0.9785 - val_loss: 0.0805 - val_accuracy: 0.9765
Epoch 6/10
422/422 [==============================] - 2s 5ms/step - loss: 0.0837 - accuracy: 0.9754 - val_loss: 0.0890 - val_accuracy: 0.9762
Epoch 7/10
422/422 [==============================] - 2s 5ms/step - loss: 0.0891 - accuracy: 0.9739 - val_loss: 0.0892 - val_accuracy: 0.9752
Epoch 8/10
422/

In [ ]:
_, model_for_pruning_accuracy = model_for_pruning.evaluate(
   test_images, test_labels, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Pruned test accuracy:', model_for_pruning_accuracy)

Baseline test accuracy: 0.9779000282287598
Pruned test accuracy: 0.9739000201225281


In [ ]:
%tensorboard --logdir={logdir}

<IPython.core.display.Javascript object>

## Create 3x smaller models from pruning

Чтобы сохранить уменьшенную версию модели надо сделать два действия
* `tfmot.sparsity.keras.strip_pruning` -- удаляет все `tf.Variable` которые нужны только во время треннинга
* и применить какой-нибудь алгоритм сжатия типа `gzip`, так как большинство весов стало нулевыми, то он будет эффективно сжимать

In [ ]:
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)

_, pruned_keras_file = tempfile.mkstemp('.h5')
tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)
print('Saved pruned Keras model to:', pruned_keras_file)

<ipython-input-10-11ad47b980ba>:4: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  tf.keras.models.save_model(model_for_export, pruned_keras_file, include_optimizer=False)


Saved pruned Keras model to: /tmp/tmp_0g536ss.h5


Конвертим модельку в `TFlite`




In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
pruned_tflite_model = converter.convert()

_, pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(pruned_tflite_file, 'wb') as f:
  f.write(pruned_tflite_model)

print('Saved pruned TFLite model to:', pruned_tflite_file)

Saved pruned TFLite model to: /tmp/tmpu0aoumf1.tflite


In [ ]:
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

In [ ]:
print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned Keras model: %.2f bytes" % (get_gzipped_model_size(pruned_keras_file)))
print("Size of gzipped pruned TFlite model: %.2f bytes" % (get_gzipped_model_size(pruned_tflite_file)))

Size of gzipped baseline Keras model: 78162.00 bytes
Size of gzipped pruned Keras model: 25886.00 bytes
Size of gzipped pruned TFlite model: 24809.00 bytes


## Create a 10x smaller model from combining pruning and quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_for_export)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_and_pruned_tflite_model = converter.convert()

_, quantized_and_pruned_tflite_file = tempfile.mkstemp('.tflite')

with open(quantized_and_pruned_tflite_file, 'wb') as f:
  f.write(quantized_and_pruned_tflite_model)

print('Saved quantized and pruned TFLite model to:', quantized_and_pruned_tflite_file)

print("Size of gzipped baseline Keras model: %.2f bytes" % (get_gzipped_model_size(keras_file)))
print("Size of gzipped pruned and quantized TFlite model: %.2f bytes" % (get_gzipped_model_size(quantized_and_pruned_tflite_file)))

Saved quantized and pruned TFLite model to: /tmp/tmpyvy9a169.tflite
Size of gzipped baseline Keras model: 78162.00 bytes
Size of gzipped pruned and quantized TFlite model: 8168.00 bytes
